<a href="https://colab.research.google.com/github/vtecftwy/metagenomics/blob/refactor_cnn_virus/nbs/2_02_EC_train_cnn_virus_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test CNN_Virus code and refactor

Original code is [here](https://github.com/MaHaoran627/CNN_Virus)

In [1]:
import numpy as np
import os
import sys
from pathlib import Path

%load_ext autoreload
%autoreload 2

In [2]:
try:
    from google.colab import drive
    ON_COLAB = True
    print('Running on colab')
    p2repo = Path('repo')
    if not p2repo.is_dir():
        !git clone -b refactor_cnn_virus https://github.com/vtecftwy/metagenomics.git repo
    
    drive.mount('/content/gdrive')
    p2drive = Path('/content/gdrive/MyDrive/Metagenonics/')
    p2data =  p2drive / 'CNN_Virus_data'
    p2src = p2repo / 'src'
except:
    ON_COLAB = False
    print('Runnng locally')
    pdata = Path('data/cnn_virus')
    p2src = Path('src')

if p2src.is_dir():
    sys.path.insert(1, str(p2src))
    print(f"Added following path to system: {p2src}")
else:
    print(f"There is no directory {p2src}")

Running on colab
Cloning into 'repo'...
remote: Enumerating objects: 128, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 128 (delta 62), reused 114 (delta 52), pack-reused 0
Receiving objects: 100% (128/128), 12.03 MiB | 29.68 MiB/s, done.
Resolving deltas: 100% (62/62), done.
Mounted at /content/gdrive
Added following path to system: repo/src


# Preprocessing data

In [3]:
from preprocessing import get_learning_weights, get_params_50mer, get_kmer_from_50mer
from preprocessing import DataGenerator_from_50mer

from architecture import build_model
from utils import TrainingExperiment

In [4]:
#path for the training file
filepath_train= p2data /"50mer_training"
assert filepath_train.is_file()
#path for the validating file
filepath_val= p2data / "50mer_validating"
assert filepath_val.is_file()
#path for the learning weights file
filepath_weights=p2data / "weight_of_classes"
assert filepath_weights.is_file()

In [5]:
d_nucl={"A":0,"C":1,"G":2,"T":3,"N":4}

f_matrix,f_labels,f_pos=get_kmer_from_50mer(filepath_train, max_seqs=1_000_000)
f_matrix_val,f_labels_val,f_pos_val=get_kmer_from_50mer(filepath_val)

params = get_params_50mer()
d_weights=get_learning_weights(filepath_weights)

training_generator = DataGenerator_from_50mer(f_matrix, f_labels, f_pos, **params)
val_generator = DataGenerator_from_50mer(f_matrix_val, f_labels_val, f_pos_val, **params)

# Build model and review it

In [6]:
model=build_model()
model.summary()

Creating Model
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 50, 5)]      0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 50, 512)      13312       ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 50, 512)     2048        ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 25, 512)      0           ['batch_normal

# Run Experiments

In [10]:
exp = TrainingExperiment(
    name='EC_1_1Mseq', 
    train_ds=training_generator, 
    val_ds=val_generator,
    p2drive=p2drive
)

Experiment Description updated in /content/gdrive/MyDrive/Metagenonics/saved/cnn_virus/EC_1_1Mseq-220831-1730-UTC--description.txt
Experiment EC_1_1Mseq ready.
Models will be saved in /content/gdrive/MyDrive/Metagenonics/saved/cnn_virus


In [12]:
exp.run_epochs(lr=1e-4, epochs=10, patience=5)

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Experiment Description updated in /content/gdrive/MyDrive/Metagenonics/saved/cnn_virus/EC_1_1Mseq-220831-1730-UTC--description.txt
Experiment Description updated in /content/gdrive/MyDrive/Metagenonics/saved/cnn_virus/EC_1_1Mseq-220831-1730-UTC--description.txt
Saved model and losses in /content/gdrive/MyDrive/Metagenonics/saved/cnn_virus as 
 - EC_1_1Mseq--220831-1747-UTC-model-002
 - EC_1_1Mseq--220831-1747-UTC-losses-002.json
